## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = 5*10**4

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 44.8 ms ± 157 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 25082.092251677885

Time taken by reduction operation using numpy.sum(): 91.8 µs ± 946 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.sum(): 25082.092251677703 
 
Time taken by reduction operation using numpy.ndarray.sum(): 72.2 µs ± 202 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 25082.092251677703


Ejercicio 3.2a) Multiprocessing

In [2]:
import numpy as np
import multiprocessing
import time

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

def split_array(array, num_parts):
    """
    Split the array into `num_parts` subarrays.
    """
    size = array.size
    ranges = []
    
    if num_parts == 1:
        return [array]  # No splitting, return the whole array as one part.
    
    # Split the array into parts
    step = size // num_parts
    for i in range(num_parts):
        start = i * step
        # Ensure that the last part gets the remaining elements if not exactly divisible
        end = (i + 1) * step if i != num_parts - 1 else size
        ranges.append(array[start:end])
    
    return ranges


if __name__ == "__main__":
    # Establecer el tamaño del array directamente
    value = 5 * 10**4  # Puedes cambiar este valor a lo que necesites
    X = np.random.rand(value)
    
    print(f"Tamaño del array: {X.size}\n")
    
    # Suma secuencial
    print("Suma secuencial usando reduc_operation")
    start_time = time.time()  
    result = reduc_operation(X)  
    end_time = time.time()  
    print(f"Resultado: {result}")  # Muestro el resultado de la suma
    print(f"Tiempo: {end_time - start_time:.6f} segundos\n")  # Muestro el tiempo
    
    # Para cada número de procesos: 1, 2 y 4
    for num_procs in [1, 2, 4]:
        print(f"Paralelismo usando {num_procs} procesos")
        
        # Dividimos el array en partes
        array_parts = split_array(X, num_procs)
        
        # Medir tiempo utilizando multiprocessing Pool y map
        start_time = time.time()  
        
        with multiprocessing.Pool(processes=num_procs) as pool:
            # Realizamos el cálculo paralelo usando map
            partial_results = pool.map(reduc_operation, array_parts)  # Ejecutamos reduc_operation en paralelo para cada parte
            total_sum = sum(partial_results)  # Sumamos los resultados parciales
        
        end_time = time.time() 
        
        print(f"Resultado con {num_procs} procesos: {total_sum}")
        print(f"Tiempo con {num_procs} procesos: {end_time - start_time:.6f} segundos\n")


Tamaño del array: 50000

Suma secuencial usando reduc_operation
Resultado: 25012.121000171774
Tiempo: 0.049609 segundos

Paralelismo usando 1 procesos
Resultado con 1 procesos: 25012.121000171774
Tiempo con 1 procesos: 0.150711 segundos

Paralelismo usando 2 procesos
Resultado con 2 procesos: 25012.12100017199
Tiempo con 2 procesos: 0.076221 segundos

Paralelismo usando 4 procesos
Resultado con 4 procesos: 25012.12100017185
Tiempo con 4 procesos: 0.093564 segundos



Ejercicio 3.2b) NUMBA

In [6]:
import numpy as np
from numba import njit, prange
import time

# Función original
def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Versión optimizada con Numba utilizando @njit
@njit
def reduc_operation_njit(A):
    """Compute the sum of the elements of Array A using Numba's njit for optimization."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Versión optimizada con Numba y paralelización utilizando @njit(parallel=True)
@njit(parallel=True)
def reduc_operation_njit_parallel(A):
    """Compute the sum of the elements of Array A using Numba's njit and parallelism."""
    s = 0
    # Usamos prange para paralelizar el bucle
    for i in prange(A.size):
        s += A[i]
    return s

# Secuencial

value = 5*10**4
X = np.random.rand(value)

# Secuencial
print("Suma secuencial usando la función original")
tiempo = %timeit -r 2 -o -q reduc_operation(X)
print("Time taken by reduction operation using a function:", tiempo)
print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")

# Usando la versión optimizada con Numba
print("Suma usando la versión optimizada con @njit")
tiempo = %timeit -r 2 -o -q reduc_operation_njit(X)
print("Time taken by reduction operation using @njit:", tiempo)
print(f"And the result using @njit is: {reduc_operation_njit(X)}\n")

# Usando la versión optimizada con Numba y paralelización
print("Suma usando la versión optimizada con @njit(parallel=True)")
tiempo = %timeit -r 2 -o -q reduc_operation_njit_parallel(X)
print("Time taken by reduction operation using @njit(parallel=True):", tiempo)
print(f"And the result using @njit(parallel=True) is: {reduc_operation_njit_parallel(X)}\n")


Suma secuencial usando la función original
Time taken by reduction operation using a function: 46.2 ms ± 848 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 24953.105604108925

Suma usando la versión optimizada con @njit
Time taken by reduction operation using @njit: 214 µs ± 9.55 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result using @njit is: 24953.105604108925

Suma usando la versión optimizada con @njit(parallel=True)
Time taken by reduction operation using @njit(parallel=True): 2.14 ms ± 524 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result using @njit(parallel=True) is: 24953.105604108947



3.2 c) En el fichero de salida:
Time taken by reduction operation using a function: 46.2 ms ± 88 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 24950.685616279792

Time taken by reduction operation using numpy.sum(): 104 µs ± 400 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.sum(): 24950.685616279643 
 
Time taken by reduction operation using numpy.ndarray.sum(): 75 µs ± 17.8 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 24950.685616279643
Tamaño del array: 5000000

Suma secuencial usando reduc_operation
Resultado: 2499520.8855190203
Tiempo: 4.640532 segundos

Paralelismo usando 1 procesos
Resultado con 1 procesos: 2499520.8855190203
Tiempo con 1 procesos: 4.780688 segundos

Paralelismo usando 2 procesos
Resultado con 2 procesos: 2499520.8855191553
Tiempo con 2 procesos: 2.619068 segundos

Paralelismo usando 4 procesos
Resultado con 4 procesos: 2499520.8855191246
Tiempo con 4 procesos: 1.395307 segundos

Suma secuencial usando la función original
Time taken by reduction operation using a function: 4.5 s ± 11.9 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 2500515.0086682923

Suma usando la versión optimizada con @njit
Time taken by reduction operation using @njit: 20.8 ms ± 16.5 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result using @njit is: 2500515.0086682923

Suma usando la versión optimizada con @njit(parallel=True)
Time taken by reduction operation using @njit(parallel=True): 9.75 ms ± 3.76 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result using @njit(parallel=True) is: 2500515.008668029


3.2 d) Explicación de los resultados: Observamos que numpy.sum en cada caso sigue siendo la opción más eficiente en términos, para la reducción de arrays grandes, dada su implementación altamente optimizada.
El uso de paralelización con Numba ofrece una mejora de rendimiento, especialmente en sistemas con múltiples núcleos de CPU; pero observamos que cuando se aplico sobre el valor de 5*10**7 el tiempo de ejecución disminuyò al aumentar el número de procesos que cuando se aplico sobre 5*10**4, ya que el paralelismo tiene la capacidad de funcionar mejor con una mayor cantidad de datos. 
Para tareas específicas donde no se pueda usar NumPy, la optimización con @njit(parallel=True) es una excelente alternativa, ofreciendo un rendimiento cercano al de NumPy, pero como se meciono anteriormente si el manejo de datos es muy grande, sino basta con el uso del decorador njit.